# Description

This is an implementation of a Furniture Recommender for an area in a selected place in a room using Contrastive Learning. CL is a powerful form of machine learning that teaches a model to differentiate between similar and dissimilar data points by contrasting them against each other. This form of machine learning is ideal for this use case.

# Packages

In [14]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import Dataset, DataLoader, random_split

# Model class

In [7]:
class ContrastiveFurnEncoder(nn.Module):

    def __init__(self, embedding_dim=256):
        super().__init__()

        # encoder for room description
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.text_projector = nn.Sequential(
            nn.Linear(in_features=768, out_features=512), # 768 since it relates to BERT's output
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=embedding_dim)
        )
        
        self.area_encoder = nn.Sequential(
            nn.Linear(in_features=10, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=embedding_dim)
        )

        self.furniture_encoder = nn.Sequential(
            nn.Linear(10, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )
    
    def encode_room(self, text_id, text_mask, area_features):

        bert_output = self.bert(input_ids=text_id, attention_mask=text_mask)
        text_features = self.text_projector(bert_output.pooler_output)

        area_features = self.area_encoder(area_features)

        room_embedding = text_features + area_features
        return torch.nn.functional.normalize(room_embedding, dim=-1)
    
    def encode_furniture(self, furniture_features):
        f_embedding = self.furniture_encoder(furniture_features)
        return torch.nn.functional.normalize(f_embedding, dim=-1)



# Loss function class: NTXentLoss

Normalized Temperature-scaled Cross Entropy Loss is a common loss function used in Contrastive Learning. It assists in training the model to recognize similar data points and distinguish dissimilar ones, allowing it to learn without labelled data.

In [3]:
class NTXentLoss(nn.Module):

    def __init__(self, t=0.07):
        super().__init__()
        self.t = t
    
    def forward(self, room_embeddings, furniture_embeddings):

        similiarity_matrix = torch.mm(room_embeddings, furniture_embeddings.t())/self.t

        labels = torch.arange(similiarity_matrix.size(0)).to(similiarity_matrix.device)

        loss_room = torch.nn.functional.cross_entropy(similiarity_matrix, labels)
        loss_furniture = torch.nn.functional.cross_entropy(similiarity_matrix.t(), labels)

        return (loss_room+loss_furniture)*0.5

# Dataset Class

In [4]:
class FurnitureRecomDataset(Dataset):

    def __init__(self, room_desc, area_feature, furniture_features, positive_pairs):
        self.room_descriptions = room_desc
        self.area_features = area_feature
        self.furniture_features = furniture_features
        self.positive_pairs = positive_pairs
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __getitem__(self, index):
        room_index, furniture_index = self.positive_pairs[index]
        room_desc = self.room_descriptions[room_index]

        encoded = self.tokenizer(room_desc, padding=True, truncation=True, return_tensor = "pt")

        text_ids = encoded['input_ids'].squeeze(0)
        text_mask = encoded['attention_mask'].squeeze(0)

        return {
            'text_ids':text_ids,
            'text_mask':text_mask,
            'room_desc': self.room_descriptions[room_index],
            'area_features': self.area_features[room_index],
            'furniture_features': self.furniture_features[furniture_index]
        }
    
    def __len__(self):
        return len(self.positive_pairs)

# Training the model

### Preparations

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [12]:
model = ContrastiveFurnEncoder().to(device=device)

### Training function

In [13]:
def train_contrastive_model(model, train_loader, num_epochs, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = NTXentLoss()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            
            text_ids = batch['text_ids'].to(device)
            text_mask = batch['text_mask'].to(device)
            area_features = batch['area_features'].to(device)
            furniture_features = batch['furniture_features'].to(device)
            
            
            room_embeddings = model.encode_room(text_ids, text_mask, area_features)
            furniture_embeddings = model.encode_furniture(furniture_features)
            
            
            loss = criterion(room_embeddings, furniture_embeddings)
            total_loss+=loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")